In [81]:
# imports modules
from src.paths import LOCAL_RAW_DATA_PATH, LOCAL_PROCESSED_DATA_PATH, LOCAL_MODELS_PATH
from tqdm._tqdm_notebook import tqdm_notebook
from transformers import pipeline
import pandas as pd
import xgboost
import pickle

tqdm_notebook.pandas()

# loads data
df_path = LOCAL_PROCESSED_DATA_PATH / 'pretrain_dataset_20211013.pkl'
df = pd.read_pickle(df_path)

c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
# loads model
model = pipeline('zero-shot-classification')

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

In [4]:
# tests model
text = 'I wanna take a cab at 8 pm'
labels = ['Taxi', 'Appointment', 'Trip', 'Game']
model(text, labels, multi_label=True)

{'sequence': 'I wanna take a cab at 8 pm',
 'labels': ['Taxi', 'Trip', 'Game', 'Appointment'],
 'scores': [0.8777869939804077,
  0.6453558802604675,
  0.05611748248338699,
  0.04601442068815231]}

In [56]:
# sets columns width
pd.set_option('max_colwidth', 100) 

In [73]:
sample = df.full_text.sample(15, random_state=0)
sample

31069     RT @ggreenwald: Please, please avoid having dinner with your family on Thanksgiving.\n\nStay at ...
64870                                                     @TomBilyeu @MoralisWeb3 which ones do you watch? :)
66644                                                                      @DanaAela Ok.\n\nWhy are you here?
76115                                                                        @nicholasilechie Sarcasm is hard
78109                                                         Are we in bizarro world https://t.co/CyukYsaqco
44310                                                                                       @tottycoys 🙏🙏🙏🙏🙏🙏
23336                     @A_Sovereign_Man @MichaelRippe @rekodi_i Yes, that is what I am. Give me a break. 🙄
109491    "Bitcoin could have already facilitated billions of dollars worth of censorship-resistant value ...
126286                                                                       @LHeilpern @CFN_network Crush it
67837     

In [69]:
txt = sample[77159]
print(txt)

@RookieXBT @KeyboardMonkey3 @CryptoCobain Please donate to a local DV shelter, they are hurting 🤝


In [82]:
labels = [
    'Crypto',
    'Bitcoin',
]

In [83]:
# tests model
model(txt, labels, multi_label=True)

{'sequence': '@RookieXBT @KeyboardMonkey3 @CryptoCobain Please donate to a local DV shelter, they are hurting 🤝',
 'labels': ['Crypto', 'Bitcoin'],
 'scores': [0.7257638573646545, 0.424002468585968]}

In [85]:
df['zsc_classes'] = df.full_text.progress_apply(lambda x: model(x, labels, multi_label=True))

  0%|          | 0/92071 [00:00<?, ?it/s]

In [97]:
mask = df.zsc_classes.progress_apply(lambda x: max(x['scores']) > 0.65)
df[mask].head()

  0%|          | 0/92071 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-97-4f5d5efa2718>", line 2, in <module>
    df[mask]
  File "c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\pandas\core\frame.py", line 2897, in __getitem__
    return self._getitem_bool_array(key)
  File "c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\pandas\core\frame.py", line 2951, in _getitem_bool_array
    return self._take_with_is_copy(indexer, axis=0)
  File "c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\pandas\core\generic.py", line 3363, in _take_with_is_copy
    result = self.take(indices=indices, axis=axis)
  File "c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\pandas\core\generic.py", line 3351, in take
    indices, axis=self._get_block_ma

TypeError: object of type 'NoneType' has no len()

In [ ]:
mask.sum()

In [87]:
df['zsc_classes'].to_pickle(LOCAL_PROCESSED_DATA_PATH / 'zsc_raw_classes_BTC.pkl')